In [1]:
# loading the database credentials from the .gitignore protected password file
import pandas as pd
import psycopg2 as pg
pass_file = "/mnt/data/mvesc/pgpass"
f = open(pass_file, 'r')
with open(pass_file, 'r') as f:
   passinfo = f.read()
passinfo = passinfo.strip().split(':')
f.close()
host_address = passinfo[0]
user_name = passinfo[2]
name_of_database = passinfo[3]
user_password = passinfo[4]
conn = pg.connect(host=host_address, database=name_of_database, user=user_name, password=user_password)

# notebook options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#### Tables Zhe is in charge of
list_of_table_names = ['AIRScores', 'ASQ_Preschool', 'ActScores',
                       'DIBELSv2', 'HSGrades', 'OAAOGT', 'PARCC',
                       'StarEL', 'StarMath', 'StarRead', 'TerraNova']

In [3]:
# functions explore a table
def get_column_names(table, connection):
    """
    Get column names of a table 
    
    :param pg.extensions.connection object connection: sql connection
    :param string table: table name in the database
    :rtype: list 
    """
    temp_table = pd.read_sql("select * FROM \"%s\" limit 1" % table, connection)
    return list(temp_table.columns)

def read_table_to_df(table_name, connection, maxStudentLookup=1e6):
    """ from Jackie
    Takes a table name as input and returns postgres table as pandas data frame.
    
    :param string table_name: Name of table to read in
    :param psycopg2.connection connection: POSTGRESQL connection object for Python
    :return: a Pandas dataframe object containing the desired table
    :rtype: Pandas.dataframe
    """
    sql_query = "SELECT * FROM \"" + table_name + "\";"
    data = pd.read_sql(sql_query, connection)
    return data

def different_columns(table1, table2, connection):
    """ find different columns names of two tables
    
    :param string table1: name of table 1
    :param string table2: name of table 2
    :param pg.connection connection: sql connection
    :return: a list with 2 sets of different columns names in 2 tables
    :rtype: list[set, set]
    """
    columns1 = get_column_names(table1, connection)
    columns2 = get_column_names(table2, connection)
    diff_col1 = set(columns1) - set(columns2)
    diff_col2 = set(columns2) - set(columns1)
    return([diff_col1, diff_col2])

def generate_colnames(schoolyear='1415'):
    """ generate the colnames based on the shcool year, e.g. 1415
    
    :para string schoolyear: school year, e.g. 1415
    :return: a list of columns names
    :rtype: list[string]
    """
    columns_name_base = ['StudentLookup', 'CURRENT_GRADE', 'HANDICAP', 'HANDICAP_DESC', 'DISADVANTAGEMENT', 'DISADVANTAGEMENT_DESC', 'DATE_OF_BIRTH', 'STUDENT_STATUS', 'STUDENT_STATUS_DESC', 'GENDER', 'RACIAL_ETHNIC_DESC', 'LIMITED_ENGLISH', 'GIFTED', 'ADMISSION_DATE', 'PERCENT_ATTEND', 'DAYS_IN_ATTENDANCE', 'DAYS_ABSENT', 'TARDY', 'PERCENT_ATTEND', 'DAYS_IN_ATTENDANCE', 'DAYS_ABSENT', 'TARDY', 'DISC_INCIDENTS', 'IN_SCHOOL_SUSP', 'OUT_OF_SCHOOL_SUSP', 'EXPUL', 'District', 'School']
    current_year=int(schoolyear[-2:]) # last 2 characters of a table name
    year_append1 = "_20"+str(current_year-2)+"_"+str(current_year-1)
    year_append2 = "_20"+str(current_year-3)+"_"+str(current_year-2)
    year_append = [year_append1]*4 + [year_append2]*4 + [year_append1]*4
    col_append = ['']*14 + year_append + ['']*2
    columns = [columns_name_base[i]+col_append[i] for i in range(len(columns_name_base))]
    return(columns)

### Overall Test Scores Exploration
We're interested in seeing what the different test scores look like. So far, we've just done a brief look at each test, which we discuss below.

#### Leftover Tasks
- We need to merge the student lookup numbers to get district-level (or school-level analysis)
- Figure out what the 'Ctr' column means
- Get test-specific means and averages for the MVESC area -- to compare how students are to their peers.

#### AIRScores
This was only given in "Fall2015" and "Spring2015" in our data. We believe it was started to be given in Ohio in the 2014-15 school year. AIR Scores is an end-of-course exam in multiple subjects for students.

#### ASQ_Preschool
This table is unclear what's exactly going on. It contains one score for a student lookup number and if they passed. There's a column named 'RecCounter', which is unclear.

#### DIBELSv2
It looks like this test is only given in K - 6 (no higher?)

In [10]:
list_of_table_names
list_of_table_names[-4]
df = read_table_to_df(list_of_table_names[-4], conn)

['AIRScores',
 'ASQ_Preschool',
 'ActScores',
 'DIBELSv2',
 'HSGrades',
 'OAAOGT',
 'PARCC',
 'StarEL',
 'StarMath',
 'StarRead',
 'TerraNova']

'StarEL'

In [11]:
df.head()
df.dtypes

,StudentLookup,ID,ExportGrade,Gender,Ethnicity,CourseName,DateTaken,AssessmentGrade,GP,SS,...,VO,CO,SC,VD,AP,CW,EN,PC,PR,NCE
0,NaN,7872,1,M,,READING,2012-05-16 12:48:00,1,1.85,786,...,76.0,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30535.0,7873,1,F,,READING,2012-01-19 13:43:00,1,1.46,801,...,79.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30535.0,7874,1,F,,READING,2012-05-18 13:53:00,1,1.85,836,...,87.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30540.0,7875,1,,,Grade 1,2012-01-09 13:02:00,1,1.42,675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30544.0,7876,1,M,,READING,2011-09-19 11:40:00,1,1.06,685,...,58.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


StudentLookup                          float64
ID                                       int64
ExportGrade                             object
Gender                                  object
Ethnicity                               object
CourseName                              object
DateTaken                       datetime64[ns]
AssessmentGrade                         object
GP                                     float64
SS                                       int64
LiteracySkillsClassification            object
GR                                     float64
GK                                     float64
PA                                     float64
PH                                     float64
SA                                     float64
VO                                     float64
CO                                     float64
SC                                     float64
VD                                     float64
AP                                     float64
CW           

In [33]:
df.groupby('NCE').agg({'NCE': 'count'}).sort_index(ascending=False).head(20)

,NCE
NCE,
99.0,41
93.3,48
89.6,58
86.9,45
84.6,45
82.7,42
81.1,49
79.6,52
78.2,57


In [16]:
df.sort_index?

In [25]:
for col_name in [x for x in air_scores.columns if x not in ['StudentLookup', 'Ctr', 'Overallscaledscore']]:
    air_scores.groupby(col_name).agg({col_name: 'count'})

,EnrolledGrade
EnrolledGrade,
,1144
03,382
04,1124
05,1039
06,1037
07,40
08,1120
09,1411
10,2176


,AdministrationDate
AdministrationDate,
Fall2015,1525
Spring2015,11940


,GiftedandTalented
GiftedandTalented,
,11162
N,2195
Y,108


,504Plan
504Plan,
,496
N,12687
Y,282


,IEP
IEP,
,2345
N,9536
Y,1584


...

,EOY tested
EOY tested,
,278
N,452
Y,12735


,Subscore1
Subscore1,
,10
Above Std,5005
At Std,5051
Below Std,3399


,Subscore2
Subscore2,
,8
Above Std,5469
At Std,4476
Below Std,3512


,Subscore3
Subscore3,
,8
Above Std,4835
At Std,5228
Below Std,3394


,Subscore4
Subscore4,
,11201
Above Std,797
At Std,958
Below Std,509


In [21]:
air_scores.describe()

/home/jgutman/env/lib/python3.4/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,StudentLookup,Ctr,GraduationPoints
count,12338.000000,13465.000000,6162.000000
mean,16932.026585,8563.983810,2.781240
std,9483.079019,5031.821931,1.138595
min,20.000000,1.000000,1.000000
25%,NaN,3367.000000,NaN
50%,NaN,9545.000000,NaN
75%,NaN,12911.000000,NaN
max,34235.000000,16277.000000,5.000000


In [27]:
list_of_table_names[1]
asq = read_table_to_df(list_of_table_names[1], conn)
asq.head()
asq.dtypes

'ASQ_Preschool'

,StudentLookup,RecCounter,Score,Plevel
0,33022.0,1,30,Pass
1,NaN,2,5,Pass
2,33032.0,3,25,Pass
3,33034.0,4,50,Pass
4,33039.0,5,0,Pass


StudentLookup    float64
RecCounter         int64
Score             object
Plevel            object
dtype: object

In [29]:
for col_name in [x for x in asq.columns if x not in ['StudentLookup', 'RecCounter']]:
    asq.groupby(col_name).agg({col_name: 'count'})

,Score
Score,
***,14
0,154
000,4
005,16
010,12
015,13
020,11
025,13
030,10


,Plevel
Plevel,
,14
Fail,266
Pass,1410


In [34]:
list_of_table_names[3]
df = read_table_to_df(list_of_table_names[3], conn)
df.head()
df.dtypes

'DIBELSv2'

,StudentLookup,Ctr,Grade,Test,Score_B,Status_B,National_Percentile_B,Score_M,Status_M,National_Percentile_M,Score_E,Status_E,National_Percentile_E
0,26850.0,680,01,NWF-CLS,51.0,Low Risk,83.0,NaN,,NaN,NaN,,NaN
1,17061.0,681,01,NWF-CLS,33.0,Low Risk,54.0,NaN,,NaN,43.0,Emerging,18.0
2,17070.0,682,01,NWF-CLS,31.0,Low Risk,50.0,NaN,,NaN,NaN,,NaN
3,28307.0,683,01,NWF-CLS,23.0,Some Risk,32.0,NaN,,NaN,34.0,Emerging,8.0
4,NaN,684,01,NWF-CLS,12.0,At Risk,12.0,NaN,,NaN,38.0,Emerging,12.0


StudentLookup            float64
Ctr                        int64
Grade                     object
Test                      object
Score_B                  float64
Status_B                  object
National_Percentile_B    float64
Score_M                  float64
Status_M                  object
National_Percentile_M    float64
Score_E                  float64
Status_E                  object
National_Percentile_E    float64
dtype: object

In [35]:
for col_name in [x for x in df.columns if x in ['Grade', 'Status_E']]:
    df.groupby(col_name).agg({col_name: 'count'})

,Grade
Grade,
01,1140
02,1059
03,852
04,943
05,21
06,25
1,1803
2,1192
3,291


,Status_E
Status_E,
,9905
At Risk,6
Deficit,18
Emerging,32
Established,88
Low Risk,10
Some Risk,8
